In [49]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from scipy.stats import t, chi2
import plotly.graph_objects as go

**System Variables**

In [50]:
# Event codes
ARRIVAL=0
DEPARTURE=1
event_codes = {ARRIVAL: "ARRIVAL", DEPARTURE: "DEPARTURE"}

# Patient status codes
PENDING=0
ADMITTED=1
RELOCATED=2
REJECTED=3
DISCHARGED=4
status_codes = {PENDING: "PENDING", ADMITTED: "ADMITTED", RELOCATED: "RELOCATED", REJECTED: "REJECTED"}

# Simulation parameters
SIM_TIME = 31   # Simulation time in days

# Ward parameters
WARDS = dict(
    names = ['A', 'B', 'C', 'D', 'E', 'F'],
    lams = [14.5, 11.0, 8.0, 6.5, 5.0, 13.0],
    mu_invs = [2.9, 4.0, 4.5, 1.4, 3.9, 2.2],
    urgency_points = [7, 5, 2, 10, 5, 0],
)

NUM_WARDS = len(WARDS['names'])

RELOCATION_PROBABILITIES = np.array([
    [0.0, 0.05, 0.10, 0.05, 0.80, 0.00],
    [0.2, 0, 0.50, 0.15, 0.15, 0.00],
    [0.30, 0.20, 0, 0.20, 0.30, 0.00],
    [0.35, 0.30, 0.05, 0, 0.3, 0.00],
    [0.20, 0.10, 0.60 ,0.10, 0, 0.00],
    [0.20, 0.20, 0.20, 0.20, 0.20 ,0]
])

**Penalty Computation**

In [51]:
def penalty(urgency_points, counts):
    return np.dot((counts[RELOCATED] + counts[REJECTED]), urgency_points)

**Simulation Setup**

In [52]:
def simulatation_setup(sim_time=SIM_TIME):

    # Patients
    patients = dict( ID=[], type=[], ward=[], status=[], U1=[], U2=[], burn_in=[] )

    # Precompute arrival and departure times for each ward
    events = dict( event_type=[], patient_ID=[], time=[] )
    patient_id = 0
    for i, (lam, mu_inv) in enumerate(zip(WARDS['lams'], WARDS['mu_invs'])):
        
        # Sample patients for ward
        clock = 0
        while clock < sim_time:

            # Sample arrival time
            U1, U2 = np.random.uniform(size=2)
            clock += -np.log(U1)/lam

            # Add patient to event list if arrival time is before simulation end
            if clock <= sim_time:
                # Event
                events['time'] += [clock, clock-np.log(U2)*mu_inv]
                events['patient_ID'] += [patient_id]*2
                events['event_type'] += [ARRIVAL, DEPARTURE]

                # Patient
                patients['ID'] += [patient_id]
                patients['type'] += [i]
                patients['ward'] += [None]
                patients['status'] += [PENDING]
                patients['U1'] += [U1]
                patients['U2'] += [U2]
                patients['burn_in'] += [False]

                # Update patient ID
                patient_id += 1


    # Sort events by time
    idx = np.argsort(events['time'])
    for key in events.keys():
        events[key] = [events[key][i] for i in idx]

    return patients, events

**Inspect the Event List**

In [53]:
# Setup simulation
patients, events = simulatation_setup()

# Create dataframes and map codes to names
patient_df = pd.DataFrame(patients); patient_df['status'] = patient_df['status'].map(status_codes)
patient_df['type'] = patient_df['type'].map({i: WARDS['names'][i] for i in range(NUM_WARDS)})
event_df = pd.DataFrame(events); event_df['event_type'] = event_df['event_type'].map(event_codes)

# Print dataframes
print("Patients:")
print(patient_df.head().to_string(index=False), "\n")
print("Events:")
print(event_df.head().to_string(), "\n")

Patients:
 ID type ward  status       U1       U2  burn_in
  0    A None PENDING 0.412093 0.029485    False
  1    A None PENDING 0.536539 0.084112    False
  2    A None PENDING 0.565522 0.128828    False
  3    A None PENDING 0.422998 0.049675    False
  4    A None PENDING 0.439180 0.040421    False 

Events:
  event_type  patient_ID      time
0    ARRIVAL         461  0.020233
1    ARRIVAL        1211  0.037504
2    ARRIVAL         796  0.038889
3    ARRIVAL        1010  0.048537
4    ARRIVAL        1375  0.049131 



**Simulation Loop**

In [54]:
def simulate(patients, events, capacities, burn_in_period=15):

    # Clear patient status
    N = len(patients['ID'])
    patients['ward'] = [None]*N
    patients['status'] = [PENDING]*N

    # List for saving ward states
    states = []

    # Dict for saving counts
    counts = {
        ADMITTED: np.zeros(NUM_WARDS, dtype=int),
        RELOCATED: np.zeros(NUM_WARDS, dtype=int),
        REJECTED: np.zeros(NUM_WARDS, dtype=int),
    }

    # Simulate loop
    occupancy = np.zeros(NUM_WARDS, dtype=int)
    for i, (event_type, patient_id) in enumerate(zip(events['event_type'], events['patient_ID'])):

        # Check if patient is in burn-in period
        if events['time'][i] < burn_in_period:
            patients['burn_in'][patient_id] = True

        # Get patient type
        patient_type = patients['type'][patient_id]
        
        if event_type == ARRIVAL:

            # Admit patient
            if occupancy[patient_type] < capacities[patient_type]:
                ward = patients['type'][patient_id]
                status = ADMITTED

            else:
                # Relocate patient (or reject if alternative ward is full)
                ward = np.random.choice(NUM_WARDS, p=RELOCATION_PROBABILITIES[patient_type])
                status = RELOCATED if occupancy[ward] < capacities[ward] else REJECTED

            # Update patient
            if status != REJECTED:
                occupancy[ward] += 1
                patients['ward'][patient_id] = ward
            patients['status'][patient_id] = status

            if events['time'][i] >= burn_in_period:
                counts[status][patient_type] += 1

        # Discharge patient
        elif patients['status'][patient_id] != REJECTED:
            occupancy[patients['ward'][patient_id]] -= 1
    
        states.append(occupancy.copy())

    events['states'] = states

    # Check if simulation ended with non-zero occupancy
    if any(occupancy != 0):
        print('Error: Simulation ended with non-zero occupancy')

    return dict(states=states, counts=counts, penalty=penalty(WARDS['urgency_points'], counts))

**Run Simulation**

In [55]:
patients, events = simulatation_setup()
capacities = [49, 28, 22, 16, 16, 34]
sim_out = simulate(patients, events, capacities)
states = sim_out['states']

fig = go.Figure()
for i, state in enumerate(np.array(events['states']).T):
    fig.add_trace(go.Scatter(x=events['time'], y=state, mode='lines', name=WARDS['names'][i]))
fig.update_layout(title='Ward occupancies', xaxis_title='Time', yaxis_title='Occupancy')
fig.show()

**Gradient Computation**

In [56]:
def compute_gradients(capacities):
    M = len(capacities)
    grads = np.zeros(M)
    n = 10

    def simulate_penalty(capacity_adjustment):
        total_penalty = 0
        for _ in range(n):
            capacities[i] += capacity_adjustment
            patients, events = simulatation_setup()
            sim_out = simulate(patients, events, capacities)
            total_penalty += sim_out['penalty']
            capacities[i] -= capacity_adjustment
        return total_penalty / n

    for i in range(M):
        p_small = simulate_penalty(-1)
        p_large = simulate_penalty(1)
        grads[i] = (p_large - p_small) / 2

    return grads


def gradient_descent(capacities, beds_to_F=34):

    penalties = np.zeros(beds_to_F+1)
    F_rel_prob = np.zeros(beds_to_F+1)

    # Realocate beds to F
    for i in tqdm(range(beds_to_F+1)):

        patients, events = simulatation_setup()
        sim_out = simulate(patients, events, capacities)
        penalties[i] = sim_out['penalty']
        n_F = sim_out['counts'][ADMITTED][-1] + sim_out['counts'][RELOCATED][-1] + sim_out['counts'][REJECTED][-1]
        F_rel_prob[i] = (sim_out['counts'][RELOCATED][-1] + sim_out['counts'][REJECTED][-1]) / n_F

        grads = compute_gradients(capacities)
        idx = np.argmax(grads[:-1])
        capacities[idx] -= 1
        capacities[-1] += 1

    return dict(capacities=capacities, penalties=penalties, F_rel_prob=F_rel_prob)   

**Optimizing Bed Allocation**

In [63]:
# # Run gradient descent for different bed totals (Takes a few minutes) 
out165 = gradient_descent([55, 40, 30, 20, 20, 0])
# capacities165 = out165['capacities']
# out170 = gradient_descent([55, 40, 30, 20, 20, 5])
# capacities170 = out170['capacities']
# out180 = gradient_descent([55, 40, 30, 20, 20, 15])
# capacities180 = out180['capacities']

capacities165 = [49, 28, 22, 16, 16, 34]
capacities170 = [48, 31, 25, 18, 14, 34]
capacities180 = [52, 35, 25, 16, 18, 34]

**Theoretical Optimal Number of Beds in F**

In [59]:
def erlangB_formula(m):
    if m == 0:
        return 1

    lam = 13; s = 2.2
    A = lam*s
    def power_div_factorial(i):
        return np.exp(i*np.log(A) - np.sum(np.log(range(1, i+1))))
    return power_div_factorial(m)/(np.sum([power_div_factorial(i) for i in range(m+1)]))

In [60]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(35), y=[erlangB_formula(m) for m in range(35)], mode='lines', name='Erlang B'))
fig.add_trace(go.Scatter(x=np.arange(35), y=out165['F_rel_prob'], mode='lines', name='Simulation'))
fig.show()

**Estimate Rates for Admission, Relocation and Rejection**

In [61]:
def estimate_rates(samples, capacities):
    num_samples = len(samples)
    rates = np.zeros((num_samples, 3, 6))
    for i in range(num_samples):
        patients, events = samples[i]
        sim_out = simulate(patients, events, capacities)
        counts = sim_out['counts']
        rates[i, 0] = counts[ADMITTED]
        rates[i, 1] = counts[RELOCATED]
        rates[i, 2] = counts[REJECTED]

    rates /= rates.sum(axis=(1, 2))[:, None, None]
    rates = rates.mean(axis=0)
    rates = pd.DataFrame(rates.T, index=WARDS['names'], columns=['ADMITTED', 'RELOCATED', 'REJECTED'])
    return rates

In [64]:
num_samples = 100
sim_setup_samples = [simulatation_setup() for _ in range(num_samples)]

for n, cap in zip([165, 170, 180], [capacities165, capacities170, capacities180]):
    rates = estimate_rates(sim_setup_samples, cap)
    print(f'{n} beds:')
    print(rates.to_string(), '\n')


165 beds:
   ADMITTED  RELOCATED  REJECTED
A  0.206546   0.015089  0.029705
B  0.101911   0.045123  0.041054
C  0.056975   0.049765  0.030304
D  0.086020   0.013872  0.011430
E  0.039891   0.023818  0.022800
F  0.213104   0.006925  0.005667 

170 beds:
   ADMITTED  RELOCATED  REJECTED
A  0.206946   0.014982  0.029413
B  0.114147   0.040889  0.033053
C  0.065029   0.045134  0.026881
D  0.096204   0.008650  0.006468
E  0.036654   0.027958  0.021897
F  0.213104   0.007289  0.005304 

180 beds:
   ADMITTED  RELOCATED  REJECTED
A  0.223540   0.011593  0.016207
B  0.128340   0.035725  0.024024
C  0.071018   0.046573  0.019453
D  0.092094   0.012602  0.006627
E  0.049888   0.021365  0.015255
F  0.213104   0.008393  0.004199 

